In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.ndimage import uniform_filter1d, gaussian_filter1d
import pandas as pd
from plotly.subplots import make_subplots


In [ ]:
# load the tire data file
# tire_data = pd.read_csv('../Data/logs/dataLog00121.TXT')
# tire_data = pd.read_csv('../Data/logs/Data71.csv')
dt = np.dtype([('time', '<u4'),
('micros', '<u4'),
('pressure', 'f4'),
('temp', 'f4'),
('aX', 'f4'),
('aY', 'f4'),
('aZ', 'f4'),
('gX', 'f4'),
('gY', 'f4'),
('gZ', 'f4'),
('aX.1', 'f4'),
('aY.1', 'f4'),
('aZ.1', 'f4'),
('dt', '<u4'),
('queue', '<u4'),])
with open('../Data/logs/Data63.bin', 'rb') as f:
    b = f.read()
# truncate to the nearest multiple of the dtype size
if len(b) % dt.itemsize != 0:
    b = b[:-(len(b) % dt.itemsize)]
data = np.frombuffer(b, dtype=dt)
data = np.sort(data, order='micros')
tire_data = pd.DataFrame(data[50:])


In [ ]:
if 'output_Hz' in tire_data.columns:
    tire_data['aX'] = tire_data['aX'] / 1000
    tire_data['aY'] = tire_data['aY'] / 1000
    tire_data['aZ'] = tire_data['aZ'] / 1000
else:
    tire_data['aX'] = tire_data['aX'] / 9.81
    tire_data['aY'] = tire_data['aY'] / 9.81
    tire_data['aZ'] = tire_data['aZ'] / 9.81
if 'aX.1' in tire_data.columns:
    tire_data['aZ.1'] = tire_data['aZ.1'] * -1

# swap x and y axes
tire_data['aX'], tire_data['aY'] = tire_data['aY'], tire_data['aX'].copy()
tire_data['aX.1'], tire_data['aY.1'] = tire_data['aY.1'], tire_data['aX.1'].copy()
tire_data['gX'], tire_data['gY'] = tire_data['gY'], tire_data['gX'].copy()
if 'micros' in tire_data.columns:
    tire_data['raw_time'] = tire_data['time']
    tire_data['time'] = tire_data['micros'] / 1000000
elif 'time' in tire_data.columns:
    tire_data['raw_time'] = tire_data['time']
    
    if 'micros' in tire_data.columns:
        tire_data['time'] = tire_data['micros'] / 1000000#np.floor(tire_data['time'] / 1000) + (np.mod(tire_data['micros'], 1000) / 1000000)
    else:
        tire_data['time'] = tire_data['time'] / 1000
else:
    tire_data['time'] = tire_data['rtcTime']
if 'aX.1' in tire_data.columns:
    cf_component = gaussian_filter1d(np.array(tire_data['aZ'] - tire_data['aZ.1']) * 9.81, 3)
else:
    cf_component = np.array(tire_data['aZ']) * 9.81
radius_est = 0.218 # m 0.218
if 'output_Hz' in tire_data.columns:
    tire_rate = gaussian_filter1d(np.array(tire_data['gX']), 3) / 180 * np.pi
else:
    tire_rate = gaussian_filter1d(np.array(tire_data['gY']), 3)
cf_from_radius = radius_est * (tire_rate)**2
contact_delta = cf_component - cf_from_radius

In [ ]:
# plot the data
fig = make_subplots(rows=6, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['aX'], name='aX'), row=1, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['aY'], name='aY'), row=1, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['aZ'], name='aZ'), row=1, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['aX.1'], name='aX.1'), row=1, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['aY.1'], name='aY.1'), row=1, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['aZ.1'], name='aZ.1'), row=1, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=cf_component, name='cf_a'), row=2, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=cf_from_radius, name='cf_r'), row=2, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=contact_delta, name='cf_delta'), row=3, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['gX'], name='gX'), row=4, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['gY'], name='gY'), row=4, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['gZ'], name='gZ'), row=4, col=1)
if 'PSI' in tire_data.columns:
    fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['PSI'], name='Pressure (absolute)'), row=5, col=1)
else:
    valid_pressure = tire_data['pressure'] > 0
    fig.add_trace(go.Scatter(x=tire_data['time'][valid_pressure], y=tire_data['pressure'][valid_pressure], name='Pressure (absolute)'), row=5, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['temp'], name='Temperature'), row=5, col=1)
fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['queue'], name='Queue'), row=5, col=1)
if 'output_Hz' in tire_data.columns:
    fig.add_trace(go.Scatter(x=tire_data['time'], y=tire_data['output_Hz'], name='Hz'), row=6, col=1)
else:
    fig.add_trace(go.Scatter(x=tire_data['time'], y=1000000/tire_data['dt'], name='Hz'), row=6, col=1) # , mode="markers"
time_dt = np.zeros(len(tire_data['time']))
time_dt[1:] = np.diff(tire_data['time'])
fig.add_trace(go.Scatter(x=tire_data['time'], y=1/time_dt, name='Hz Real'), row=6, col=1)
fig.update_layout(template="plotly_dark",title_text=f"Hi")
fig.show()


In [ ]:
np.average(contact_delta[np.abs(contact_delta) < 3])

In [ ]:
(one_revolution,) = np.where((tire_data['time'] > 48.15) & (tire_data['time'] < 48.3))
middle_time = 48.24443
tire_data['time_dt'] = tire_data['raw_time'] - tire_data['raw_time'].shift(1)
tire_data['time_dt'][0] = 0
(bad_times,) = np.where(tire_data['time_dt'] > np.ceil(tire_data['dt']/1000))
corr_times = tire_data['dt'].copy() / 1000000
corr_times[bad_times] = tire_data['time_dt'][bad_times] / 1000
tire_data['angle'] = np.cumsum(tire_data['gX'] * corr_times) * 180 / np.pi
(middle_ind,) = np.where(np.abs(tire_data['time'] - middle_time) < 0.0001)
middle_angle = tire_data['angle'][middle_ind].values[0]
angles = np.array(tire_data['angle'][one_revolution]) - middle_angle
if angles[0] > 0:
    angles *= -1
times = tire_data['time'][one_revolution] - middle_time
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=angles, y=tire_data['aX'][one_revolution]*9.81, name='aX'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=tire_data['aY'][one_revolution]*9.81, name='aY'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=tire_data['aZ'][one_revolution]*9.81, name='aZ'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=tire_data['aX.1'][one_revolution]*9.81, name='aX.1'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=tire_data['aY.1'][one_revolution]*9.81, name='aY.1'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=tire_data['aZ.1'][one_revolution]*9.81, name='aZ.1'), row=1, col=1)
delta_x = gaussian_filter1d(tire_data['aX'][one_revolution] - tire_data['aX.1'][one_revolution], 3)
fig.add_trace(go.Scatter(x=angles, y=delta_x*9.81, name='daX'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=cf_component[one_revolution], name='cf_a'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=cf_from_radius[one_revolution], name='cf_r'), row=1, col=1)
fig.add_trace(go.Scatter(x=angles, y=contact_delta[one_revolution], name='cf_delta'), row=1, col=1)
fig.update_layout(template="plotly_dark",title_text=f"Hi")
fig.update_xaxes(title_text="Tire Angle (deg After BDC)", row=1, col=1)
fig.update_yaxes(title_text="Contact Force (m/s^2)", row=1, col=1)
fig.show()

In [ ]:
initial_angle = -2.2
final_angle = 3.9
rate_at_bdc = tire_rate[middle_ind][0]
speed_at_bdc = rate_at_bdc * radius_est
total_angle = final_angle - initial_angle
total_time = total_angle / (rate_at_bdc * 180 / np.pi)
total_distance = speed_at_bdc * total_time
print(f"Speed at BDC: {speed_at_bdc:.3f} m/s")
print(f"Total Angle: {total_angle:.2f} deg")
print(f"Total Time: {total_time:.4f} s")
print(f"Total Distance: {total_distance:.4f} m")

In [ ]:
tire_data.columns

In [ ]:
tire_data

In [ ]:
tire_data[tire_data['time'] == 0]